# core

> Fill in a module description here

In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| exporti
import sqlite3
import argparse
from mcp.server.fastmcp import FastMCP
from typing import List, Tuple, Optional

In [ ]:
#| export
def init_db(db_path: str = 'demo.db') -> Tuple[sqlite3.Connection, sqlite3.Cursor]:
    """Initialize SQLite database and create people table if it doesn't exist.
    
    Args:
        db_path (str): Path to SQLite database file
        
    Returns:
        Tuple[sqlite3.Connection, sqlite3.Cursor]: Database connection and cursor
    """
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS people (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            name TEXT NOT NULL,
            age INTEGER NOT NULL,
            profession TEXT NOT NULL
        )
    ''')
    
    conn.commit()
    return conn, cursor

In [ ]:
# Test the database initialization
conn, cursor = init_db('test.db')
print("Database initialized successfully")
conn.close()

In [ ]:
#| export

# MCP Server Setup
class SQLiteMCPServer:
    """MCP Server for SQLite database operations."""
    
    def __init__(self, server_name: str = 'sqlite-demo', db_path: str = 'demo.db'):
        self.mcp = FastMCP(server_name)
        self.db_path = db_path
        self._setup_tools()
    
    def _setup_tools(self):
        """Setup MCP tools for database operations."""
        
        @self.mcp.tool()
        def add_data(query: str) -> bool:
            """Add new data to the people table using a SQL INSERT query.
            
            Args:
                query (str): SQL INSERT query following this format:
                    INSERT INTO people (name, age, profession) VALUES ('John Doe', 30, 'Engineer')
                    
            Schema:
                - name: Text field (required)
                - age: Integer field (required)  
                - profession: Text field (required)
                Note: 'id' field is auto-generated
                
            Returns:
                bool: True if data was added successfully, False otherwise
                
            Example:
                >>> query = '''
                ... INSERT INTO people (name, age, profession)
                ... VALUES ('Alice Smith', 25, 'Developer')
                ... '''
                >>> add_data(query)
                True
            """
            conn, cursor = init_db(self.db_path)
            try:
                cursor.execute(query)
                conn.commit()
                return True
            except sqlite3.Error as e:
                print(f"Error adding data: {e}")
                return False
            finally:
                conn.close()
        
        @self.mcp.tool()
        def read_data(query: str = "SELECT * FROM people") -> List[Tuple]:
            """Read data from the people table using a SQL SELECT query.
            
            Args:
                query (str, optional): SQL SELECT query. Defaults to "SELECT * FROM people".
                
            Examples:
                - "SELECT * FROM people"
                - "SELECT name, age FROM people WHERE age > 25"
                - "SELECT * FROM people ORDER BY age DESC"
                
            Returns:
                List[Tuple]: List of tuples containing the query results.
                For default query, tuple format is (id, name, age, profession)
                
            Example:
                >>> # Read all records
                >>> read_data()
                [(1, 'John Doe', 30, 'Engineer'), (2, 'Alice Smith', 25, 'Developer')]
                
                >>> # Read with custom query
                >>> read_data("SELECT name, profession FROM people WHERE age < 30")
                [('Alice Smith', 'Developer')]
            """
            conn, cursor = init_db(self.db_path)
            try:
                cursor.execute(query)
                return cursor.fetchall()
            except sqlite3.Error as e:
                print(f"Error reading data: {e}")
                return []
            finally:
                conn.close()
    
    def run(self, server_type: str = "sse"):
        """Run the MCP server.
        
        Args:
            server_type (str): Type of server to run ("sse" or "stdio")
        """
        print(f"Starting SQLite MCP Server...")
        print(f"Database: {self.db_path}")
        print(f"Server Type: {server_type}")
        self.mcp.run(server_type)

In [ ]:
# Test the MCP server setup
server = SQLiteMCPServer('test-server', 'test.db')
print("MCP Server created successfully")
print(f"Server instance: {type(server.mcp).__name__}")
print(f"Database path: {server.db_path}")

In [ ]:
#| export

# Main Function
def main():
    """Main function to run the MCP server with command line arguments."""
    parser = argparse.ArgumentParser(description="SQLite MCP Server")
    parser.add_argument(
        "--server_type", 
        type=str, 
        default="sse",
        choices=["sse", "stdio"],
        help="Type of server to run"
    )
    parser.add_argument(
        "--db_path",
        type=str,
        default="demo.db",
        help="Path to SQLite database file"
    )
    
    args = parser.parse_args()
    
    # Create and run server
    server = SQLiteMCPServer(db_path=args.db_path)
    server.run(args.server_type)

In [ ]:
# Example usage demonstration
if __name__ == "__main__":
    # Create server instance
    server = SQLiteMCPServer('demo-server', 'demo.db')
    
    # You can test the tools directly here
    # Note: In production, tools are called by MCP clients
    print("Server setup complete!")
    print("To run server: python -m llamaindex_mcp_project.server")

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()